In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download msambare/fer2013

: 

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/fer2013.zip", 'r')
zip_ref.extractall("/content")
zip_ref.close()


In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
TRAIN_DIR='/content/train'
TEST_DIR='/content/test'

In [ ]:
def createDataFrame(dir):
    image_paths = []
    labels = []

    for label in os.listdir(dir):
        label_dir = os.path.join(dir, label)
        for imagename in os.listdir(label_dir):
            image_paths.append(os.path.join(label_dir, imagename))
            labels.append(label)

        print(f"{label} completed")

    return image_paths, labels

In [ ]:
train=pd.DataFrame()
train['image'],train['label']=createDataFrame(TRAIN_DIR)

In [ ]:
print(train)


In [ ]:
test=pd.DataFrame()
test['image'],test['label']=createDataFrame(TEST_DIR)

In [ ]:
print(test)
print(test['image'])


In [ ]:
from tqdm import tqdm
from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D


In [ ]:

def extractFeatures(images):
    features=[]
    for images in tqdm(images):
         img = load_img(images, color_mode="grayscale")
         img=np.array(img)
         features.append(img)
    features=np.array(features)
    features=features.reshape(len(features),48,48,1)
    return features

In [ ]:
train_features=extractFeatures(train['image'])

In [ ]:
test_features=extractFeatures(test['image'])


In [ ]:
x_train=train_features/255.0
x_test=test_features/255.0

In [ ]:
from sklearn.preprocessing import LabelEncoder


In [ ]:
le=LabelEncoder()
le.fit(train['label'])

In [ ]:
y_train=le.transform(train['label'])
y_test=le.transform(test['label'])

In [ ]:
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [ ]:
model=Sequential()
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:

model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test))

In [ ]:
from keras.models import Sequential

In [ ]:
model_json=model.to_json()
with open("emotiondetector.json","w")as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [ ]:
from keras.models import model_from_json

In [ ]:
with open("emotiondetector.json", "r") as json_file:
    model_json = json_file.read()
    json_file.close()
    model=model_from_json(model_json)
    model.load_weights("emotiondetector.h5")


In [ ]:
label=["angry","disgust","fear","happy","neutral ","sad","surprise"]

In [37]:
def ef(image):
    img=load_img(image,color_mode="grayscale")
    features=np.array(img)
    features=features.reshape(1,48,48,1)
    return features/255.0

In [45]:
image='/content/train/angry/0.jpg'
print('original image is angry')
img=ef(image)
pred=model.predict(img)
pred_label=label[pred.argmax()]
print("model prediction is ",pred_label)

original image is angry


FileNotFoundError: [Errno 2] No such file or directory: '/content/train/angry/0.jpg'